# U-net 1

In [ ]:
!pip install albumentations --quiet
!pip install git+https://github.com/tensorflow/examples.git --quiet
!pip install tensorflow_addons --quiet
!pip install segmentation-models --quiet

     |████████████████████████████████| 631 kB 31.9 MB/s 
    Running setup.py install for tensorflow-examples ... done
  DEPRECATION: tensorflow-examples was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
     |████████████████████████████████| 1.1 MB 35.0 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from shutil import copyfile
import numpy as np
import scipy.io
from tensorflow.keras import layers
import glob
import imageio
import os
import PIL
import time
from IPython import display
import albumentations as A
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2 as cv
import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda, Activation, MaxPool2D, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from IPython.display import clear_output
from sklearn.preprocessing import LabelEncoder
from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow_addons as tfa
from keras import backend as K

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# copyfile('gdrive/My Drive/DEEP-project/Progetto_DL_finale/immagini2000.npy', 'immagini2000.npy')
# copyfile('gdrive/My Drive/DEEP-project/Progetto_DL_finale/maschere2000.npy', 'maschere2000.npy')
# copyfile('gdrive/My Drive/DEEP-project/Progetto_DL_finale/X_val.npy', 'X_val.npy')
# copyfile('gdrive/My Drive/DEEP-project/Progetto_DL_finale/y_val.npy', 'y_val.npy')

#### Train set

In [ ]:
num = 1600

In [ ]:
n_classes=19

In [ ]:
X_train = np.load('immagini2000.npy')[0:num,:,:,:]
y_train = np.array(tf.expand_dims(np.load('maschere2000.npy')[0:num,:,:],3))

FileNotFoundError: ignored

#### Categorizzare train set

In [ ]:
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))
y_train = 0
train_masks_cat = 0

#### Validation set

In [ ]:
n_classes = 19

In [ ]:
# num_val = 980
# num_val2 = np.random.permutation(np.arange(num_val))[0:200]

In [ ]:
# X_val = np.load('X_val.npy')[0:num_val,:,:,:]
# X_val = X_val/X_val.max()
# y_val = np.array(tf.expand_dims(np.load('y_val.npy')[0:num_val,:,:],3))

In [ ]:
# lista1 = []
# lista2 = []
# for i in num_val2:
#   lista1.append(X_val[i,...])
#   lista2.append(y_val[i,...])
# X_val=np.array(lista1)
# y_val=np.array(lista2)
# num_val2 = 0

#### Categorizzare validation set

In [ ]:
# train_masks_cat = to_categorical(y_val, num_classes=n_classes)
# y_val_cat = train_masks_cat.reshape((y_val.shape[0], y_val.shape[1], y_val.shape[2], n_classes))
# y_val = 0
# train_masks_cat = 0

#### Data loader

In [ ]:
batch_size = 8

In [ ]:
class DataGenerator(keras.utils.Sequence):
  def __init__(self, x_data, y_data, batch_size):
    self.x, self.y = x_data, y_data
    self.batch_size = batch_size
    self.num_batches = np.ceil(len(x_data) / batch_size)
    self.batch_idx = np.array_split(range(len(x_data)), self.num_batches)

  def __len__(self):
    return len(self.batch_idx)

  def __getitem__(self, idx):
    batch_x = self.x[self.batch_idx[idx]]
    batch_y = self.y[self.batch_idx[idx]]
    return batch_x, batch_y

In [ ]:
train_generator = DataGenerator(X_train, y_train_cat, batch_size = batch_size)
#val_generator = DataGenerator(X_val, y_val_cat, batch_size = batch_size)

### U-net

In [ ]:
def multi_unet_model(n_classes=19, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=6):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    c3 = BatchNormalization()(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    c4 = BatchNormalization()(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
    c6 = BatchNormalization()(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
    c7 = BatchNormalization()(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
    c8 = BatchNormalization()(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    c9 = BatchNormalization()(c9)
     
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    
    #NOTE: Compile the model in the main program to make it easy to test with various loss functions
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    #model.summary()
    
    return model

In [ ]:
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]

def get_model():
    return multi_unet_model(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)

In [ ]:
def focal_loss(gamma=2., alpha=.25):
	def focal_loss_fixed(y_true, y_pred):
		pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
		pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
		return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1+K.epsilon())) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))
	return focal_loss_fixed

In [ ]:
model = get_model()
model.compile(optimizer='adam', loss=focal_loss(), metrics=[tf.keras.metrics.Recall(),
                                                          tf.keras.metrics.Precision(),
                                                          tf.keras.metrics.OneHotMeanIoU(num_classes=n_classes),
                                                          'accuracy'])
#tfa.losses.SigmoidFocalCrossEntropy(), 'categorical_crossentropy', focal_loss
#model.summary()

In [ ]:
#tf.keras.utils.plot_model(model, to_file='1a_U-net.png',show_shapes=True, dpi=1028)

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='train_loss', patience=5) #val_one_hot_mean_io_u

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
# k = 87
# sample_image, sample_mask = X_val[k,:,:,:], create_mask(tf.expand_dims(y_val_cat[k,:,:,:],0)) 
k = 87
sample_image, sample_mask = X_train[k,:,:,:], create_mask(tf.expand_dims(y_train_cat[k,:,:,:],0)) 

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    if i==0:
      plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    else:
      plt.imshow(display_list[i])

    plt.axis('off')
  plt.show()

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image[:,:,0:3], sample_mask[:,:,0], create_mask(model.predict(tf.expand_dims(sample_image,0)))[:,:,0]])

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

### Training

In [ ]:
history = model.fit(x = train_generator,
                    verbose=1, 
                    epochs=50, 
                    #validation_data = val_generator,
                    shuffle = True,
                    callbacks=[callback,DisplayCallback()]
                    )

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, history.history['one_hot_mean_io_u'], 'y')
plt.plot(epochs, history.history['val_one_hot_mean_io_u'], 'r')
plt.title('Training and validation hot io_u')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, history.history['precision'], 'y')
plt.plot(epochs, history.history['val_precision'], 'r')
plt.title('Training and validation precision')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


plt.plot(epochs, history.history['recall'], 'y',)
plt.plot(epochs, history.history['val_recall'], 'r')
plt.title('Training and validation recall')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Download and upload of the model

In [ ]:
json_model = model.to_json()

In [ ]:
with open('U-net_1_aug.json', 'w') as json_file:
    json_file.write(json_model)

In [ ]:
model.save_weights('U-net_1_aug_weights.h5')

In [ ]:
!cp U-net_1_aug.json "gdrive/My Drive/dl-prog-pro/"

In [ ]:
!cp U-net_1_aug_weights.h5 "gdrive/My Drive/dl-prog-pro/"

In [ ]:
with open('U-net_1_aug.json', 'r') as json_file:
    json_savedModel = json_file.read()
net = tf.keras.models.model_from_json(json_savedModel)
net.load_weights('U-net_1_aug_weights.h5')